In [ ]:
import numpy
import networkx as nx
import copy
from numba import jit
from scipy.optimize import curve_fit

In [ ]:
@jit
def run_urn_model(neigh_dict, rho, nu, N0, APlength, t_max):
    t=0
    
    #creating a common list of concepts which will appear in the adjacent possible
    APlist = range(APlength)

    #creating Urn dictionaries with
    Urn={} #urn elements
    Urn_AP_index = 0 
    Urn_S={} #sequences of extracted elements
    Urn_D={} #set of distinct extracted elements

    #Initializing the dictionaries, the key is the node int
    for node in neigh_dict.keys():
        #initially there are N0 elements, taken from the APlist
        Urn[node]=APlist[:N0]
        #we still haven't extracted anything from the urns
        Urn_S[node]=[] 
        Urn_D[node]=set()
    
    #so the moment we have taken the first N0 elements for all the nodes
    Urn_AP_index = N0 #index of elements in the APlist
    
    #Simulation
    while t < t_max: #main simulation cycle
        if t%2000==0: print ('t', t)
        
        #All Extractions at once SYNCHRONOUS
        extracted_balls={} #temporary dictionary {node: extracted ball at t}
        
        #print ('EXTRACTIONS')
        for node in neigh_dict.keys():
            #print ('node', node)
            
            Social_urn = Urn[node] #the current urn
            #plus the others
            for nei in neigh_dict[node]:
                #summing up the urn of node with the SET of the urns of the neighbors
                Social_urn = Social_urn + list(set(Urn[nei]))
            
            #print ('My urn', Urn[node])
            #print ('Social urn', Social_urn)
            extracted_ball = np.random.choice(Social_urn)
            #print ('extracted ball', extracted_ball)
            
            #Adding it to the sequence
            Urn_S[node].append(extracted_ball)
            #Adding it to the temporary dictionary
            extracted_balls[node] = extracted_ball

        #print ('REINFORCEMENT AND EXPANSION')
        for node in neigh_dict.keys():
            #print ('node', node)
            extracted_ball = extracted_balls[node]
            #print ('extracted ball', extracted_ball)
            
            #REINFORCEMENT: adding rho additional balls of the same color to the urn
            Urn[node].extend([extracted_ball] * rho)
            #print ('reinforced urn', Urn[node])
            
            #ADJACENT POSSIBLE
            
            #if the ball was new (never extracted before)
            if extracted_ball not in Urn_D[node]:
                Urn_D[node].add(extracted_ball)
                #print ('extracted ball is NEW', extracted_ball)
                #Adding nu+1 balls to the urn, by using the index on the APlist (different new balls for each urn)
                Urn[node].extend(APlist[Urn_AP_index:(Urn_AP_index + (nu+1))])
                #updating the index
                Urn_AP_index = Urn_AP_index + (nu+1)
                #print ('extended urn', Urn[node])
        
        t += 1
        
    return Urn_S

@jit
def get_heap_from_sequence(seq):
    unique_balls = set()
    heaps_list = []
    heaps_list.append(len(unique_balls))
    for ball in seq:
        unique_balls.add(ball)
        heaps_list.append(len(unique_balls))
    return heaps_list

@jit
def run_simulation(neigh_dict, rho, nu, N0, APlength, t_max, iterations):

    heaps_mean={}
    for node in nodes_list:
        heaps_mean[node]=[]

    for i in range(iterations):
        print ('iteration', i)
        Urn_S = run_urn_model(neigh_dict, rho, nu, N0, APlength, t_max)
        heaps={}
        for node, seq in Urn_S.iteritems():
            heaps[node] = get_heap_from_sequence(seq)
        for k, v in heaps.iteritems():
            heaps_mean[k].append(v)

    for node in nodes_list:
        heaps_mean[node] = np.array(heaps_mean[node])
        heaps_mean[node] = heaps_mean[node].mean(0)

    return heaps_mean

Simulations for the fourth network of Fig. 3 - this is not in parallel, just for testing

In [ ]:
#Network
G = nx.DiGraph()
G.add_edge(0,1)
G.add_edge(1,2)
G.add_edge(2,3)
G.add_edge(1,3)

#I don't actually need a G, just the list of neighbors:
neigh_dict={n: G[n].keys() for n in G.nodes()}
#And the list of nodes
nodes_list = [int(n) for n in G.nodes()]
del G

#Adjacen Possible Concept List
APlength = 100000

#Urn parameters
rho = 6
nu = 3
N0 = nu + 1 #initial number of distinct elements

#Simulation parameters
t_max = 10000
iterations = 20

In [ ]:
heaps_mean = run_simulation(neigh_dict, rho, nu, N0, APlength, t_max, iterations)

import pickle
filename="../results/Simulations_small_nets/mean_heaps_4nodes_notsym_DIR_rho%inu%i.p"%(rho, nu)
pickle.dump(heaps_mean, open(filename, "wb" ) )